In [1]:
import pandas as pd
import statsmodels.formula.api as smf

# load data
df = pd.read_csv("immigration_comments_with_period_label_updated.csv")
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# exclude 2025
filtered_df = df[df['2025'] == False].copy()

# define pairs
pairs = {
    'NYC_vs_LA': {'experimental': 'New York City', 'control': 'Los Angeles'},
    'Chicago_vs_Philadelphia': {'experimental': 'Chicago', 'control': 'Philadelphia'},
    'Denver_vs_Minneapolis': {'experimental': 'Denver', 'control': 'Minneapolis'}
}

# create post indicator
filtered_df['Post'] = (filtered_df['Period_Label'] == 'Experimental').astype(int)

# prepare subset
def prepare_subset(df, exp_city, ctrl_city, stance_vars=True):
    subset = df[df['City'].isin([exp_city, ctrl_city])].copy()
    subset['Treated'] = (subset['City'] == exp_city).astype(int)
    subset['Post'] = (subset['Period_Label'] == 'Experimental').astype(int)
    if stance_vars:
        subset['Positive'] = (subset['Stance'] == 1).astype(int)
        subset['Negative'] = (subset['Stance'] == -1).astype(int)
    return subset

# run regression
def run_did_analysis(subset, outcome_var):
    model = smf.ols(f'{outcome_var} ~ Post + Treated + Post:Treated', data=subset).fit(cov_type='HC1')
    return {
        'DiD Estimate': model.params['Post:Treated'],
        'Std Error': model.bse['Post:Treated'],
        'p-value': model.pvalues['Post:Treated'],
        '95% CI Lower': model.conf_int().loc['Post:Treated', 0],
        '95% CI Upper': model.conf_int().loc['Post:Treated', 1]
    }

# main results
results = []
for pair, cities in pairs.items():
    subset = prepare_subset(filtered_df, cities['experimental'], cities['control'])
    for outcome, label in [('Positive', 'Positive'), ('Negative', 'Negative')]:
        res = run_did_analysis(subset, outcome)
        res.update({'Pair': pair, 'Sentiment': label})
        results.append(res)

# robustness 1: mean probabilities
robust1 = []
for pair, cities in pairs.items():
    subset = prepare_subset(filtered_df, cities['experimental'], cities['control'], stance_vars=False)
    for outcome, label in [('prob_positive', 'Mean Positive Probability'), ('prob_negative', 'Mean Negative Probability')]:
        res = run_did_analysis(subset, outcome)
        res.update({'Pair': pair, 'Sentiment': label})
        robust1.append(res)

# robustness 2: exclude whales
robust2 = []
no_whales = filtered_df[filtered_df['Outlier'] == False]
for pair, cities in pairs.items():
    subset = prepare_subset(no_whales, cities['experimental'], cities['control'])
    for outcome, label in [('Positive', 'Positive (No Whales)'), ('Negative', 'Negative (No Whales)')]:
        res = run_did_analysis(subset, outcome)
        res.update({'Pair': pair, 'Sentiment': label})
        robust2.append(res)

# robustness 3: include 2025
robust3 = []
for pair, cities in pairs.items():
    subset = prepare_subset(df, cities['experimental'], cities['control'])
    for outcome, label in [('Positive', 'Positive (Including 2025)'), ('Negative', 'Negative (Including 2025)')]:
        res = run_did_analysis(subset, outcome)
        res.update({'Pair': pair, 'Sentiment': label})
        robust3.append(res)

# save csvs
pd.DataFrame(results).to_csv("did_main_results.csv", index=False)
pd.DataFrame(robust1).to_csv("did_robustness_mean_prob.csv", index=False)
pd.DataFrame(robust2).to_csv("did_robustness_no_whales.csv", index=False)
pd.DataFrame(robust3).to_csv("did_robustness_including_2025.csv", index=False)


In [4]:
import pandas as pd
import matplotlib.pyplot as plt

# load results
df = pd.read_csv("did_main_results.csv")

# sort order
pair_order = ['Chicago_vs_Philadelphia', 'NYC_vs_LA', 'Denver_vs_Minneapolis']
pair_labels = {
    'Chicago_vs_Philadelphia': 'Chicago vs Philadelphia',
    'NYC_vs_LA': 'NYC vs Los Angeles',
    'Denver_vs_Minneapolis': 'Denver vs Minneapolis'
}

# style
plt.rcParams.update({
    'font.family': 'serif',
    'axes.edgecolor': 'black',
    'axes.linewidth': 1,
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'legend.fontsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'figure.facecolor': 'white',
    'axes.facecolor': 'white',
    'savefig.facecolor': 'white',
    'savefig.edgecolor': 'white',
    'figure.dpi': 300
})

# plot function
def plot_did_bar(df, sentiment, filename):
    data = df[df['Sentiment'] == sentiment]
    data = data.set_index('Pair').loc[pair_order].reset_index()
    means = data['DiD Estimate']
    lower = data['95% CI Lower']
    upper = data['95% CI Upper']
    yerr = [means - lower, upper - means]

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.errorbar(
        x=[pair_labels[p] for p in data['Pair']],
        y=means,
        yerr=yerr,
        fmt='o',
        capsize=6,
        color='orange',
        linewidth=2,
        marker='o',
        markersize=6
    )

    ax.axhline(0, color='gray', linestyle='--')
    ax.set_ylabel("Change in Probability")
    ax.set_title(f"Difference-in-Differences Effect on {sentiment} Sentiment")
    plt.xticks(rotation=20)
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

# create both plots
plot_did_bar(df, 'Positive', 'did_effect_positive.png')
plot_did_bar(df, 'Negative', 'did_effect_negative.png')


/var/folders/zb/bx5vr4ws4vz0hjl_m9z6vkl00000gn/T/ipykernel_10907/1233062421.py:42: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  ax.errorbar(
/var/folders/zb/bx5vr4ws4vz0hjl_m9z6vkl00000gn/T/ipykernel_10907/1233062421.py:42: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  ax.errorbar(
